<a href="https://colab.research.google.com/github/RyuichiSaito1/inflation-reddit-usa/blob/main/src/tf_idf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm

In [ ]:
import pandas as pd

# Path to the TSV file
# file_path = '/content/drive/MyDrive/world-inflation/result/tsv/merged_food_results_with_year_month.tsv'
file_path = '/content/drive/MyDrive/world-inflation/result/tsv/merged_food_results_20260819_with_year_month.tsv'
# file_path = '/content/drive/MyDrive/world-inflation/result/tsv/merged_cars_results_with_year_month.tsv'
# file_path = '/content/drive/MyDrive/world-inflation/result/tsv/merged_RealEstate_results_with_year_month.tsv'
# file_path = '/content/drive/MyDrive/world-inflation/result/tsv/merged_travel_results_with_year_month.tsv'
# file_path = '/content/drive/MyDrive/world-inflation/result/tsv/merged_Frugal_results_with_year_month.tsv'

# Read the TSV file into a DataFrame
df = pd.read_csv(file_path, sep='\t')

# Display the first few rows of the DataFrame
df.tail()

In [ ]:
# food
# filtered_df = df[(df['year_month'] >= '2016-12') & (df['year_month'] <= '2020-05') & (df['inflation'] == 2)]
filtered_df = df[(df['year_month'] >= '2020-06') & (df['year_month'] <= '2022-12') & (df['inflation'] == 2)]
# cars
# filtered_df = df[(df['year_month'] >= '2018-07') & (df['year_month'] <= '2020-11') & (df['inflation'] == 2)]
# filtered_df = df[(df['year_month'] >= '2020-11') & (df['year_month'] <= '2022-12') & (df['inflation'] == 2)]
# Real Estate
# filtered_df = df[(df['year_month'] >= '2015-05') & (df['year_month'] <= '2020-07') & (df['inflation'] == 2)]
# filtered_df = df[(df['year_month'] >= '2020-08') & (df['year_month'] <= '2022-07') & (df['inflation'] == 2)]
# Travel
# filtered_df = df[(df['year_month'] >= '2019-02') & (df['year_month'] <= '2020-06') & (df['inflation'] == 2)]
# filtered_df = df[(df['year_month'] >= '2020-07') & (df['year_month'] <= '2022-12') & (df['inflation'] == 2)]
# Frugal
# filtered_df = df[(df['year_month'] >= '2015-01') & (df['year_month'] <= '2020-10') & (df['inflation'] == 2)]
# filtered_df = df[(df['year_month'] >= '2020-11') & (df['year_month'] <= '2022-04') & (df['inflation'] == 2)]
filtered_df

In [ ]:
import spacy

# Load the English tokenizer from spaCy
nlp = spacy.load("en_core_web_sm")

# Extract the 'body' column and apply stemming
stemmed_texts = filtered_df['body'].apply(lambda text: ' '.join([token.lemma_ for token in nlp(text)]))

# Display the first 10 stemmed results
for stemmed_text in stemmed_texts.head(10):
    print(stemmed_text)


In [ ]:
import re

# Clean the text by removing stopwords, numbers, and noise
cleaned_texts = stemmed_texts.apply(
    lambda text: ' '.join([token.text for token in nlp(text)
                          if not token.is_stop and token.is_alpha and not re.search(r'\d', token.text)])
)

# Convert cleaned_texts to a list of strings, as required by CountVectorizer
cleaned_texts_list = cleaned_texts.tolist()

# Display the first 10 cleaned results (stopword removed, numbers, and noise removed)
for cleaned_text in cleaned_texts_list[:10]:  # Use cleaned_texts_list instead
    print(cleaned_text)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Step 1: Create Co-occurrence Matrix
# Initialize the CountVectorizer to count word frequencies, excluding common English stopwords
vectorizer = CountVectorizer(stop_words='english')

# Fit the vectorizer on the documents and transform the text data
X = vectorizer.fit_transform(cleaned_texts_list)

# Get the list of words (features)
words = vectorizer.get_feature_names_out()

# Create the co-occurrence matrix by multiplying the term-document matrix with its transpose
co_occurrence_matrix = (X.T * X).toarray()

# Convert the co-occurrence matrix into a DataFrame for better readability
co_occurrence_df = pd.DataFrame(co_occurrence_matrix, index=words, columns=words)
print("Co-occurrence Matrix:")
print(co_occurrence_df)

In [ ]:
# # Step 2: Create TF-IDF Matrix

# # List of words to exclude
exclude_words = ['food', 'like', 'price', 'cost', 'inflation', 'deflation', 'expensive', 'cheap', 'purchase', 'sale']
# # exclude_words = ['like', 'price', 'cost', 'inflation', 'deflation', 'expensive', 'cheap', 'purchase', 'sale']
# # exclude_words = ['real', 'real', 'like', 'price', 'cost', 'inflation', 'deflation', 'expensive', 'cheap', 'purchase', 'sale']
# exclude_words = [
#     'travel', 'like', 'price', 'cost', 'inflation', 'deflation', 'expensive', 'cheap', 'purchase', 'sale',
#     'america', 'united states', 'usa', 'the us', 'u.s.', 'stateside', 'across the states',
#     'alabama', 'alaska', 'arizona', 'arkansas', 'california', 'colorado', 'connecticut', 'delaware', 'florida',
#     'georgia', 'hawaii', 'idaho', 'illinois', 'indiana', 'iowa', 'kansas', 'kentucky', 'louisiana', 'maine',
#     'maryland', 'massachusetts', 'michigan', 'minnesota', 'mississippi', 'missouri', 'montana', 'nebraska',
#     'nevada', 'new hampshire', 'new jersey', 'new mexico', 'new york', 'north carolina', 'north dakota', 'ohio',
#     'oklahoma', 'oregon', 'pennsylvania', 'rhode island', 'south carolina', 'south dakota', 'tennessee', 'texas',
#     'utah', 'vermont', 'virginia', 'washington', ' DC ', 'west virginia', 'wisconsin', 'wyoming',
#     'new york', 'ny', 'nyc', 'los angeles', 'chicago', 'houston', 'phoenix', 'philadelphia', 'san antonio', 'san diego',
#     'dallas', 'san jose', 'austin', 'jacksonville', 'fort worth', 'columbus', 'indianapolis', 'charlotte', 'san francisco', 'seattle',
#     'nashville', 'denver', 'oklahoma city', 'el paso', 'boston', 'portland', 'las vegas', 'vegas', 'detroit', 'memphis', 'louisville',
#     'baltimore', 'milwaukee', 'albuquerque', 'tucson', 'fresno', 'sacramento', 'kansas city', 'mesa', 'atlanta', 'omaha',
#     'colorado springs', 'raleigh', 'long beach', 'virginia beach', 'miami', 'oakland', 'minneapolis', 'tulsa', 'bakersfield',
#     'wichita', 'arlington', 'aurora', 'tampa', 'new orleans', 'cleveland', 'honolulu', 'anaheim', 'lexington', 'stockton',
#     'corpus christi', 'henderson', 'riverside', 'newark', 'st. paul', 'santa ana', 'cincinnati', 'irvine', 'orlando', 'pittsburgh',
#     'st. louis', 'greensboro', 'jersey city', 'anchorage', 'lincoln', 'plano', 'durham', 'buffalo', 'chandler', 'chula vista',
#     'toledo', 'madison', 'gilbert', 'reno', 'fort wayne', 'north las vegas', 'st. petersburg', 'lubbock', 'irving', 'laredo',
#     'winston-salem', 'chesapeake', 'glendale', 'garland', 'scottsdale', 'norfolk', 'boise', 'fremont', 'spokane', 'santa clarita',
#     'baton rouge', 'richmond', 'hialeah',
#     'grand canyon', 'yellowstone', 'hollywood', 'niagara', 'disney world', 'yosemite', 'central park',
#     'amtrak', 'greyhound', 'interstate', 'york', 'san', 'los'
# ]
# exclude_words = ['frugal', 'like', 'price', 'cost', 'inflation', 'deflation', 'expensive', 'cheap', 'purchase', 'sale']

# Initialize the TfidfVectorizer to calculate term frequency-inverse document frequency (TF-IDF)
tfidf_vectorizer = TfidfVectorizer(stop_words=exclude_words, ngram_range=(2, 2))

# Fit the vectorizer on the documents and transform the text data
X_tfidf = tfidf_vectorizer.fit_transform(cleaned_texts_list)

# Get the list of words (features) for TF-IDF
tfidf_words = tfidf_vectorizer.get_feature_names_out()

# Convert the TF-IDF matrix into a DataFrame for better readability
tfidf_df = pd.DataFrame(X_tfidf.toarray(), index=cleaned_texts_list, columns=tfidf_words)
print("TF-IDF Matrix:")
print(tfidf_df)


food before COVID

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np

# List of words to exclude
# food
exclude_words = ['food', 'like', 'price', 'cost', 'inflation', 'deflation', 'expensive', 'cheap', 'purchase', 'sale']
# car
# exclude_words = ['like', 'price', 'cost', 'inflation', 'deflation', 'expensive', 'cheap', 'purchase', 'sale']
# Real Estate
# exclude_words = ['real', 'estate', 'like', 'price', 'cost', 'inflation', 'deflation', 'expensive', 'cheap', 'purchase', 'sale']
# travel
# exclude_words = [
#     'travel', 'like', 'price', 'cost', 'inflation', 'deflation', 'expensive', 'cheap', 'purchase', 'sale',
#     'america', 'united states', 'usa', 'the us', 'u.s.', 'stateside', 'across the states',
#     'alabama', 'alaska', 'arizona', 'arkansas', 'california', 'colorado', 'connecticut', 'delaware', 'florida',
#     'georgia', 'hawaii', 'idaho', 'illinois', 'indiana', 'iowa', 'kansas', 'kentucky', 'louisiana', 'maine',
#     'maryland', 'massachusetts', 'michigan', 'minnesota', 'mississippi', 'missouri', 'montana', 'nebraska',
#     'nevada', 'new hampshire', 'new jersey', 'new mexico', 'new york', 'north carolina', 'north dakota', 'ohio',
#     'oklahoma', 'oregon', 'pennsylvania', 'rhode island', 'south carolina', 'south dakota', 'tennessee', 'texas',
#     'utah', 'vermont', 'virginia', 'washington', ' DC ', 'west virginia', 'wisconsin', 'wyoming',
#     'new york', 'ny', 'nyc', 'los angeles', 'chicago', 'houston', 'phoenix', 'philadelphia', 'san antonio', 'san diego',
#     'dallas', 'san jose', 'austin', 'jacksonville', 'fort worth', 'columbus', 'indianapolis', 'charlotte', 'san francisco', 'seattle',
#     'nashville', 'denver', 'oklahoma city', 'el paso', 'boston', 'portland', 'las vegas', 'vegas', 'detroit', 'memphis', 'louisville',
#     'baltimore', 'milwaukee', 'albuquerque', 'tucson', 'fresno', 'sacramento', 'kansas city', 'mesa', 'atlanta', 'omaha',
#     'colorado springs', 'raleigh', 'long beach', 'virginia beach', 'miami', 'oakland', 'minneapolis', 'tulsa', 'bakersfield',
#     'wichita', 'arlington', 'aurora', 'tampa', 'new orleans', 'cleveland', 'honolulu', 'anaheim', 'lexington', 'stockton',
#     'corpus christi', 'henderson', 'riverside', 'newark', 'st. paul', 'santa ana', 'cincinnati', 'irvine', 'orlando', 'pittsburgh',
#     'st. louis', 'greensboro', 'jersey city', 'anchorage', 'lincoln', 'plano', 'durham', 'buffalo', 'chandler', 'chula vista',
#     'toledo', 'madison', 'gilbert', 'reno', 'fort wayne', 'north las vegas', 'st. petersburg', 'lubbock', 'irving', 'laredo',
#     'winston-salem', 'chesapeake', 'glendale', 'garland', 'scottsdale', 'norfolk', 'boise', 'fremont', 'spokane', 'santa clarita',
#     'baton rouge', 'richmond', 'hialeah',
#     'grand canyon', 'yellowstone', 'hollywood', 'niagara', 'disney world', 'yosemite', 'central park',
#     'amtrak', 'greyhound', 'interstate', 'san', 'york'
# ]
# Frugal
# exclude_words = ['frugal', 'like', 'price', 'cost', 'inflation', 'deflation', 'expensive', 'cheap', 'purchase', 'sale']

# Initialize the TfidfVectorizer to calculate term frequency-inverse document frequency (TF-IDF)
# Use ngram_range=(1, 2) to consider both unigrams (1-word) and bigrams (2-words)
tfidf_vectorizer = TfidfVectorizer(stop_words=exclude_words, ngram_range=(1, 2))

# Fit the vectorizer on the documents and transform the text data
X_tfidf = tfidf_vectorizer.fit_transform(cleaned_texts_list)

# Get the list of words (features) for TF-IDF
tfidf_words = tfidf_vectorizer.get_feature_names_out()

# Convert the TF-IDF matrix into a DataFrame for better readability
tfidf_df = pd.DataFrame(X_tfidf.toarray(), index=cleaned_texts_list, columns=tfidf_words)

# Function to get top n words for each n-gram type (1-gram and 2-gram)
def get_top_n_grams(tfidf_df, n=10):
    # Sum the TF-IDF scores across all documents for each n-gram
    sum_tfidf = tfidf_df.sum(axis=0)

    # Sort the n-grams by their sum of TF-IDF values in descending order
    sorted_grams = sum_tfidf.sort_values(ascending=False)

    # Return the top n n-grams
    return sorted_grams.head(n)

# Filter unigrams (1-word) and bigrams (2-words)
unigrams = [word for word in tfidf_words if len(word.split()) == 1]
bigrams = [word for word in tfidf_words if len(word.split()) == 2]

# Get the top 10 unigrams (1-gram)
top_1grams = get_top_n_grams(tfidf_df[unigrams], n=15)
print("Top 15 Unigrams:")
print(top_1grams)

# Get the top 10 bigrams (2-gram)
top_2grams = get_top_n_grams(tfidf_df[bigrams], n=15)
print("\nTop 15 Bigrams:")
print(top_2grams)


Top 15 Unigrams:
look          143.422522
good          131.225539
pricey         73.566501
think          73.119475
eat            71.785299
place          70.814055
pretty         67.808561
buy            67.017478
worth          66.710840
use            66.628557
meat           66.495964
restaurant     65.199364
try            64.921129
time           64.413207
live           63.462105
dtype: float64

Top 15 Bigrams:
look delicious    21.100400
look good         19.692833
lobster roll      14.718989
bit pricey        14.612192
grocery store     14.234269
ice cream         12.884045
year ago          12.101452
shake shack       11.954780
pretty good       11.923791
little pricey     11.416855
look amazing      11.294659
want try          10.499790
hard find         10.175283
kobe beef          9.907465
taste good         9.263085
dtype: float64


food after COVID

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np

# List of words to exclude
# food
exclude_words = ['food', 'like', 'price', 'cost', 'inflation', 'deflation', 'expensive', 'cheap', 'purchase', 'sale']
# car
# exclude_words = ['like', 'price', 'cost', 'inflation', 'deflation', 'expensive', 'cheap', 'purchase', 'sale']
# Real Estate
# exclude_words = ['real', 'estate', 'like', 'price', 'cost', 'inflation', 'deflation', 'expensive', 'cheap', 'purchase', 'sale']
# travel
# exclude_words = [
#     'travel', 'like', 'price', 'cost', 'inflation', 'deflation', 'expensive', 'cheap', 'purchase', 'sale',
#     'america', 'united states', 'usa', 'the us', 'u.s.', 'stateside', 'across the states',
#     'alabama', 'alaska', 'arizona', 'arkansas', 'california', 'colorado', 'connecticut', 'delaware', 'florida',
#     'georgia', 'hawaii', 'idaho', 'illinois', 'indiana', 'iowa', 'kansas', 'kentucky', 'louisiana', 'maine',
#     'maryland', 'massachusetts', 'michigan', 'minnesota', 'mississippi', 'missouri', 'montana', 'nebraska',
#     'nevada', 'new hampshire', 'new jersey', 'new mexico', 'new york', 'north carolina', 'north dakota', 'ohio',
#     'oklahoma', 'oregon', 'pennsylvania', 'rhode island', 'south carolina', 'south dakota', 'tennessee', 'texas',
#     'utah', 'vermont', 'virginia', 'washington', ' DC ', 'west virginia', 'wisconsin', 'wyoming',
#     'new york', 'ny', 'nyc', 'los angeles', 'chicago', 'houston', 'phoenix', 'philadelphia', 'san antonio', 'san diego',
#     'dallas', 'san jose', 'austin', 'jacksonville', 'fort worth', 'columbus', 'indianapolis', 'charlotte', 'san francisco', 'seattle',
#     'nashville', 'denver', 'oklahoma city', 'el paso', 'boston', 'portland', 'las vegas', 'vegas', 'detroit', 'memphis', 'louisville',
#     'baltimore', 'milwaukee', 'albuquerque', 'tucson', 'fresno', 'sacramento', 'kansas city', 'mesa', 'atlanta', 'omaha',
#     'colorado springs', 'raleigh', 'long beach', 'virginia beach', 'miami', 'oakland', 'minneapolis', 'tulsa', 'bakersfield',
#     'wichita', 'arlington', 'aurora', 'tampa', 'new orleans', 'cleveland', 'honolulu', 'anaheim', 'lexington', 'stockton',
#     'corpus christi', 'henderson', 'riverside', 'newark', 'st. paul', 'santa ana', 'cincinnati', 'irvine', 'orlando', 'pittsburgh',
#     'st. louis', 'greensboro', 'jersey city', 'anchorage', 'lincoln', 'plano', 'durham', 'buffalo', 'chandler', 'chula vista',
#     'toledo', 'madison', 'gilbert', 'reno', 'fort wayne', 'north las vegas', 'st. petersburg', 'lubbock', 'irving', 'laredo',
#     'winston-salem', 'chesapeake', 'glendale', 'garland', 'scottsdale', 'norfolk', 'boise', 'fremont', 'spokane', 'santa clarita',
#     'baton rouge', 'richmond', 'hialeah',
#     'grand canyon', 'yellowstone', 'hollywood', 'niagara', 'disney world', 'yosemite', 'central park',
#     'amtrak', 'greyhound', 'interstate', 'san', 'york'
# ]
# Frugal
# exclude_words = ['frugal', 'like', 'price', 'cost', 'inflation', 'deflation', 'expensive', 'cheap', 'purchase', 'sale']

# Initialize the TfidfVectorizer to calculate term frequency-inverse document frequency (TF-IDF)
# Use ngram_range=(1, 2) to consider both unigrams (1-word) and bigrams (2-words)
tfidf_vectorizer = TfidfVectorizer(stop_words=exclude_words, ngram_range=(1, 2))

# Fit the vectorizer on the documents and transform the text data
X_tfidf = tfidf_vectorizer.fit_transform(cleaned_texts_list)

# Get the list of words (features) for TF-IDF
tfidf_words = tfidf_vectorizer.get_feature_names_out()

# Convert the TF-IDF matrix into a DataFrame for better readability
tfidf_df = pd.DataFrame(X_tfidf.toarray(), index=cleaned_texts_list, columns=tfidf_words)

# Function to get top n words for each n-gram type (1-gram and 2-gram)
def get_top_n_grams(tfidf_df, n=10):
    # Sum the TF-IDF scores across all documents for each n-gram
    sum_tfidf = tfidf_df.sum(axis=0)

    # Sort the n-grams by their sum of TF-IDF values in descending order
    sorted_grams = sum_tfidf.sort_values(ascending=False)

    # Return the top n n-grams
    return sorted_grams.head(n)

# Filter unigrams (1-word) and bigrams (2-words)
unigrams = [word for word in tfidf_words if len(word.split()) == 1]
bigrams = [word for word in tfidf_words if len(word.split()) == 2]

# Get the top 10 unigrams (1-gram)
top_1grams = get_top_n_grams(tfidf_df[unigrams], n=15)
print("Top 15 Unigrams:")
print(top_1grams)

# Get the top 10 bigrams (2-gram)
top_2grams = get_top_n_grams(tfidf_df[bigrams], n=15)
print("\nTop 15 Bigrams:")
print(top_2grams)


Top 15 Unigrams:
look          79.397912
good          66.175000
use           43.273804
meat          39.153459
eat           36.914857
think         36.711093
pricey        36.236954
time          36.221291
place         33.670440
restaurant    33.498952
buy           33.345404
way           33.013601
lobster       32.959377
worth         32.565316
live          31.108129
dtype: float64

Top 15 Bigrams:
look good         12.940388
lobster roll      10.114863
look delicious     9.221660
grocery store      8.973331
year ago           8.609246
look great         7.706228
look amazing       7.515160
bit pricey         6.681978
hard find          5.399762
hot dog            4.858838
little pricey      4.800015
high quality       4.790216
new york           4.500146
short rib          4.343338
pretty good        4.102249
dtype: float64


Cars before COVID

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np

# List of words to exclude
# car
exclude_words = ['like', 'price', 'cost', 'inflation', 'deflation', 'expensive', 'cheap', 'purchase', 'sale']
# Real Estate
# exclude_words = ['real', 'estate', 'like', 'price', 'cost', 'inflation', 'deflation', 'expensive', 'cheap', 'purchase', 'sale']

# Initialize the TfidfVectorizer to calculate term frequency-inverse document frequency (TF-IDF)
# Use ngram_range=(1, 2) to consider both unigrams (1-word) and bigrams (2-words)
tfidf_vectorizer = TfidfVectorizer(stop_words=exclude_words, ngram_range=(1, 2))

# Fit the vectorizer on the documents and transform the text data
X_tfidf = tfidf_vectorizer.fit_transform(cleaned_texts_list)

# Get the list of words (features) for TF-IDF
tfidf_words = tfidf_vectorizer.get_feature_names_out()

# Convert the TF-IDF matrix into a DataFrame for better readability
tfidf_df = pd.DataFrame(X_tfidf.toarray(), index=cleaned_texts_list, columns=tfidf_words)

# Function to get top n words for each n-gram type (1-gram and 2-gram)
def get_top_n_grams(tfidf_df, n=10):
    # Sum the TF-IDF scores across all documents for each n-gram
    sum_tfidf = tfidf_df.sum(axis=0)

    # Sort the n-grams by their sum of TF-IDF values in descending order
    sorted_grams = sum_tfidf.sort_values(ascending=False)

    # Return the top n n-grams
    return sorted_grams.head(n)

# Filter unigrams (1-word) and bigrams (2-words)
unigrams = [word for word in tfidf_words if len(word.split()) == 1]
bigrams = [word for word in tfidf_words if len(word.split()) == 2]

# Get the top 10 unigrams (1-gram)
top_1grams = get_top_n_grams(tfidf_df[unigrams], n=15)
print("Top 15 Unigrams:")
print(top_1grams)

# Get the top 10 bigrams (2-gram)
top_2grams = get_top_n_grams(tfidf_df[bigrams], n=15)
print("\nTop 15 Bigrams:")
print(top_2grams)


Top 15 Unigrams:
car       196.102963
buy        70.604764
new        67.964061
year       61.801178
look       61.105670
people     59.970622
think      59.434780
use        58.251591
drive      57.197795
want       54.505507
high       53.545882
good       50.425899
time       47.036306
lot        45.567389
know       44.088570
dtype: float64

Top 15 Bigrams:
new car       22.721342
year old      15.208491
sport car     12.700061
buy new       12.249622
use car       12.071384
buy car       10.846983
brand new     10.810440
year ago       9.570674
old car        9.305020
people buy     9.021110
car year       8.767848
luxury car     8.006060
base model     7.462739
car look       6.827017
car worth      6.735325
dtype: float64


Cars after COVID

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np

# List of words to exclude
# car
exclude_words = ['like', 'price', 'cost', 'inflation', 'deflation', 'expensive', 'cheap', 'purchase', 'sale']
# Real Estate
# exclude_words = ['real', 'estate', 'like', 'price', 'cost', 'inflation', 'deflation', 'expensive', 'cheap', 'purchase', 'sale']

# Initialize the TfidfVectorizer to calculate term frequency-inverse document frequency (TF-IDF)
# Use ngram_range=(1, 2) to consider both unigrams (1-word) and bigrams (2-words)
tfidf_vectorizer = TfidfVectorizer(stop_words=exclude_words, ngram_range=(1, 2))

# Fit the vectorizer on the documents and transform the text data
X_tfidf = tfidf_vectorizer.fit_transform(cleaned_texts_list)

# Get the list of words (features) for TF-IDF
tfidf_words = tfidf_vectorizer.get_feature_names_out()

# Convert the TF-IDF matrix into a DataFrame for better readability
tfidf_df = pd.DataFrame(X_tfidf.toarray(), index=cleaned_texts_list, columns=tfidf_words)

# Function to get top n words for each n-gram type (1-gram and 2-gram)
def get_top_n_grams(tfidf_df, n=10):
    # Sum the TF-IDF scores across all documents for each n-gram
    sum_tfidf = tfidf_df.sum(axis=0)

    # Sort the n-grams by their sum of TF-IDF values in descending order
    sorted_grams = sum_tfidf.sort_values(ascending=False)

    # Return the top n n-grams
    return sorted_grams.head(n)

# Filter unigrams (1-word) and bigrams (2-words)
unigrams = [word for word in tfidf_words if len(word.split()) == 1]
bigrams = [word for word in tfidf_words if len(word.split()) == 2]

# Get the top 10 unigrams (1-gram)
top_1grams = get_top_n_grams(tfidf_df[unigrams], n=15)
print("Top 15 Unigrams:")
print(top_1grams)

# Get the top 10 bigrams (2-gram)
top_2grams = get_top_n_grams(tfidf_df[bigrams], n=15)
print("\nTop 15 Bigrams:")
print(top_2grams)


Top 15 Unigrams:
car        200.507860
buy         76.074557
new         72.703472
people      69.125604
year        68.889828
use         60.048779
drive       59.936499
high        58.821536
think       56.870891
look        56.169809
want        55.118786
time        50.533413
good        49.437796
vehicle     49.309129
pay         45.887642
dtype: float64

Top 15 Bigrams:
new car         26.687530
use car         19.210675
year ago        15.003095
sport car       14.992450
buy car         13.183247
buy new         12.940419
year old        12.879925
brand new       11.677048
people buy      10.547999
luxury car       8.501804
car car          8.152328
car year         8.057843
old car          7.963277
electric car     7.538813
car market       7.506092
dtype: float64


RealEstate before COVID

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np

# List of words to exclude
# cars
# exclude_words = ['like', 'price', 'cost', 'inflation', 'deflation', 'expensive', 'cheap', 'purchase', 'sale']
# Real Estate
exclude_words = ['real', 'estate', 'like', 'price', 'cost', 'inflation', 'deflation', 'expensive', 'cheap', 'purchase', 'sale']

# Initialize the TfidfVectorizer to calculate term frequency-inverse document frequency (TF-IDF)
# Use ngram_range=(1, 2) to consider both unigrams (1-word) and bigrams (2-words)
tfidf_vectorizer = TfidfVectorizer(stop_words=exclude_words, ngram_range=(1, 2))

# Fit the vectorizer on the documents and transform the text data
X_tfidf = tfidf_vectorizer.fit_transform(cleaned_texts_list)

# Get the list of words (features) for TF-IDF
tfidf_words = tfidf_vectorizer.get_feature_names_out()

# Convert the TF-IDF matrix into a DataFrame for better readability
tfidf_df = pd.DataFrame(X_tfidf.toarray(), index=cleaned_texts_list, columns=tfidf_words)

# Function to get top n words for each n-gram type (1-gram and 2-gram)
def get_top_n_grams(tfidf_df, n=10):
    # Sum the TF-IDF scores across all documents for each n-gram
    sum_tfidf = tfidf_df.sum(axis=0)

    # Sort the n-grams by their sum of TF-IDF values in descending order
    sorted_grams = sum_tfidf.sort_values(ascending=False)

    # Return the top n n-grams
    return sorted_grams.head(n)

# Filter unigrams (1-word) and bigrams (2-words)
unigrams = [word for word in tfidf_words if len(word.split()) == 1]
bigrams = [word for word in tfidf_words if len(word.split()) == 2]

# Get the top 10 unigrams (1-gram)
top_1grams = get_top_n_grams(tfidf_df[unigrams], n=15)
print("Top 15 Unigrams:")
print(top_1grams)

# Get the top 10 bigrams (2-gram)
top_2grams = get_top_n_grams(tfidf_df[bigrams], n=15)
print("\nTop 15 Bigrams:")
print(top_2grams)


Top 15 Unigrams:
house       242.176073
home        206.740487
market      155.139378
buy         151.638467
year        150.861808
high        139.554748
property    132.952090
area        132.643980
pay         129.690492
offer       121.659441
sell        120.687299
look        120.330552
rent        119.200908
want        115.951669
time        106.300133
dtype: float64

Top 15 Bigrams:
buy house         44.545499
interest rate     34.193197
buy home          26.893986
year ago          25.726018
bay area          24.413026
property taxis    21.122941
sell house        20.770665
hot market        17.785955
house sell        17.588880
good luck         17.283839
single family     15.936882
seller market     15.619880
long term         15.505065
sq ft             14.894357
look house        14.758237
dtype: float64


RealEstate after COVID

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np

# List of words to exclude
# cars
# exclude_words = ['like', 'price', 'cost', 'inflation', 'deflation', 'expensive', 'cheap', 'purchase', 'sale']
# Real Estate
exclude_words = ['real', 'estate', 'like', 'price', 'cost', 'inflation', 'deflation', 'expensive', 'cheap', 'purchase', 'sale']

# Initialize the TfidfVectorizer to calculate term frequency-inverse document frequency (TF-IDF)
# Use ngram_range=(1, 2) to consider both unigrams (1-word) and bigrams (2-words)
tfidf_vectorizer = TfidfVectorizer(stop_words=exclude_words, ngram_range=(1, 2))

# Fit the vectorizer on the documents and transform the text data
X_tfidf = tfidf_vectorizer.fit_transform(cleaned_texts_list)

# Get the list of words (features) for TF-IDF
tfidf_words = tfidf_vectorizer.get_feature_names_out()

# Convert the TF-IDF matrix into a DataFrame for better readability
tfidf_df = pd.DataFrame(X_tfidf.toarray(), index=cleaned_texts_list, columns=tfidf_words)

# Function to get top n words for each n-gram type (1-gram and 2-gram)
def get_top_n_grams(tfidf_df, n=10):
    # Sum the TF-IDF scores across all documents for each n-gram
    sum_tfidf = tfidf_df.sum(axis=0)

    # Sort the n-grams by their sum of TF-IDF values in descending order
    sorted_grams = sum_tfidf.sort_values(ascending=False)

    # Return the top n n-grams
    return sorted_grams.head(n)

# Filter unigrams (1-word) and bigrams (2-words)
unigrams = [word for word in tfidf_words if len(word.split()) == 1]
bigrams = [word for word in tfidf_words if len(word.split()) == 2]

# Get the top 10 unigrams (1-gram)
top_1grams = get_top_n_grams(tfidf_df[unigrams], n=15)
print("Top 15 Unigrams:")
print(top_1grams)

# Get the top 10 bigrams (2-gram)
top_2grams = get_top_n_grams(tfidf_df[bigrams], n=15)
print("\nTop 15 Bigrams:")
print(top_2grams)


Top 15 Unigrams:
house     163.578109
home      146.224377
market    110.672748
year      107.328061
buy       106.374423
people     90.402172
high       89.368830
rate       87.152895
offer      86.206788
sell       81.365060
pay        75.598666
area       71.994751
want       71.636574
rent       71.185480
think      68.297565
dtype: float64

Top 15 Bigrams:
interest rate       38.193469
buy house           32.042560
year ago            19.712706
buy home            18.907245
housing market      13.764943
house sell          13.012088
sell house          12.884586
new construction    12.352712
offer ask           11.990217
property taxis      11.802241
home sell           11.282815
month ago           10.642864
new home             9.996990
bay area             9.866553
long term            9.838507
dtype: float64


travel before COVID

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np

# List of words to exclude
# car
# exclude_words = ['like', 'price', 'cost', 'inflation', 'deflation', 'expensive', 'cheap', 'purchase', 'sale']
# Real Estate
# exclude_words = ['real', 'estate', 'like', 'price', 'cost', 'inflation', 'deflation', 'expensive', 'cheap', 'purchase', 'sale']
# travel
exclude_words = [
    'travel', 'like', 'price', 'cost', 'inflation', 'deflation', 'expensive', 'cheap', 'purchase', 'sale',
    'america', 'united states', 'usa', 'the us', 'u.s.', 'stateside', 'across the states',
    'alabama', 'alaska', 'arizona', 'arkansas', 'california', 'colorado', 'connecticut', 'delaware', 'florida',
    'georgia', 'hawaii', 'idaho', 'illinois', 'indiana', 'iowa', 'kansas', 'kentucky', 'louisiana', 'maine',
    'maryland', 'massachusetts', 'michigan', 'minnesota', 'mississippi', 'missouri', 'montana', 'nebraska',
    'nevada', 'new hampshire', 'new jersey', 'new mexico', 'new york', 'north carolina', 'north dakota', 'ohio',
    'oklahoma', 'oregon', 'pennsylvania', 'rhode island', 'south carolina', 'south dakota', 'tennessee', 'texas',
    'utah', 'vermont', 'virginia', 'washington', ' DC ', 'west virginia', 'wisconsin', 'wyoming',
    'new york', 'ny', 'nyc', 'los angeles', 'chicago', 'houston', 'phoenix', 'philadelphia', 'san antonio', 'san diego',
    'dallas', 'san jose', 'austin', 'jacksonville', 'fort worth', 'columbus', 'indianapolis', 'charlotte', 'san francisco', 'seattle',
    'nashville', 'denver', 'oklahoma city', 'el paso', 'boston', 'portland', 'las vegas', 'vegas', 'detroit', 'memphis', 'louisville',
    'baltimore', 'milwaukee', 'albuquerque', 'tucson', 'fresno', 'sacramento', 'kansas city', 'mesa', 'atlanta', 'omaha',
    'colorado springs', 'raleigh', 'long beach', 'virginia beach', 'miami', 'oakland', 'minneapolis', 'tulsa', 'bakersfield',
    'wichita', 'arlington', 'aurora', 'tampa', 'new orleans', 'cleveland', 'honolulu', 'anaheim', 'lexington', 'stockton',
    'corpus christi', 'henderson', 'riverside', 'newark', 'st. paul', 'santa ana', 'cincinnati', 'irvine', 'orlando', 'pittsburgh',
    'st. louis', 'greensboro', 'jersey city', 'anchorage', 'lincoln', 'plano', 'durham', 'buffalo', 'chandler', 'chula vista',
    'toledo', 'madison', 'gilbert', 'reno', 'fort wayne', 'north las vegas', 'st. petersburg', 'lubbock', 'irving', 'laredo',
    'winston-salem', 'chesapeake', 'glendale', 'garland', 'scottsdale', 'norfolk', 'boise', 'fremont', 'spokane', 'santa clarita',
    'baton rouge', 'richmond', 'hialeah',
    'grand canyon', 'yellowstone', 'hollywood', 'niagara', 'disney world', 'yosemite', 'central park',
    'amtrak', 'greyhound', 'interstate', 'san', 'york', 'los'
]

# Initialize the TfidfVectorizer to calculate term frequency-inverse document frequency (TF-IDF)
# Use ngram_range=(1, 2) to consider both unigrams (1-word) and bigrams (2-words)
tfidf_vectorizer = TfidfVectorizer(stop_words=exclude_words, ngram_range=(1, 2))

# Fit the vectorizer on the documents and transform the text data
X_tfidf = tfidf_vectorizer.fit_transform(cleaned_texts_list)

# Get the list of words (features) for TF-IDF
tfidf_words = tfidf_vectorizer.get_feature_names_out()

# Convert the TF-IDF matrix into a DataFrame for better readability
tfidf_df = pd.DataFrame(X_tfidf.toarray(), index=cleaned_texts_list, columns=tfidf_words)

# Function to get top n words for each n-gram type (1-gram and 2-gram)
def get_top_n_grams(tfidf_df, n=10):
    # Sum the TF-IDF scores across all documents for each n-gram
    sum_tfidf = tfidf_df.sum(axis=0)

    # Sort the n-grams by their sum of TF-IDF values in descending order
    sorted_grams = sum_tfidf.sort_values(ascending=False)

    # Return the top n n-grams
    return sorted_grams.head(n)

# Filter unigrams (1-word) and bigrams (2-words)
unigrams = [word for word in tfidf_words if len(word.split()) == 1]
bigrams = [word for word in tfidf_words if len(word.split()) == 2]

# Get the top 10 unigrams (1-gram)
top_1grams = get_top_n_grams(tfidf_df[unigrams], n=15)
print("Top 15 Unigrams:")
print(top_1grams)

# Get the top 10 bigrams (2-gram)
top_2grams = get_top_n_grams(tfidf_df[bigrams], n=15)
print("\nTop 15 Bigrams:")
print(top_2grams)


/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['across', 'ana', 'angeles', 'antonio', 'baton', 'beach', 'canyon', 'carolina', 'central', 'christi', 'chula', 'city', 'clarita', 'corpus', 'dakota', 'dc', 'diego', 'disney', 'el', 'fort', 'francisco', 'grand', 'hampshire', 'island', 'jersey', 'jose', 'las', 'long', 'louis', 'mexico', 'new', 'north', 'orleans', 'park', 'paso', 'paul', 'petersburg', 'rhode', 'rouge', 'salem', 'santa', 'south', 'springs', 'st', 'states', 'the', 'united', 'us', 'vista', 'wayne', 'west', 'winston', 'world', 'worth'] not in stop_words.
  warnings.warn(


Top 15 Unigrams:
day       33.859492
flight    32.539139
city      29.374193
time      29.035391
trip      26.757787
good      25.484383
place     25.289780
car       25.237215
want      24.052174
look      22.926905
fly       22.281590
way       22.263643
think     21.734675
drive     20.494979
people    20.220639
dtype: float64

Top 15 Bigrams:
rent car         9.630757
national park    5.175079
spend day        4.751070
day trip         4.724252
round trip       4.619062
new orleans      4.430840
grand canyon     4.270544
car rental       4.199778
east coast       3.825993
road trip        3.750314
spend time       3.617164
west coast       3.572837
time year        3.516019
big city         3.189684
good luck        3.130565
dtype: float64


travel after COVID

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np

# List of words to exclude
# car
# exclude_words = ['like', 'price', 'cost', 'inflation', 'deflation', 'expensive', 'cheap', 'purchase', 'sale']
# Real Estate
# exclude_words = ['real', 'estate', 'like', 'price', 'cost', 'inflation', 'deflation', 'expensive', 'cheap', 'purchase', 'sale']
# travel
exclude_words = [
    'travel', 'like', 'price', 'cost', 'inflation', 'deflation', 'expensive', 'cheap', 'purchase', 'sale',
    'america', 'united states', 'usa', 'the us', 'u.s.', 'stateside', 'across the states',
    'alabama', 'alaska', 'arizona', 'arkansas', 'california', 'colorado', 'connecticut', 'delaware', 'florida',
    'georgia', 'hawaii', 'idaho', 'illinois', 'indiana', 'iowa', 'kansas', 'kentucky', 'louisiana', 'maine',
    'maryland', 'massachusetts', 'michigan', 'minnesota', 'mississippi', 'missouri', 'montana', 'nebraska',
    'nevada', 'new hampshire', 'new jersey', 'new mexico', 'new york', 'north carolina', 'north dakota', 'ohio',
    'oklahoma', 'oregon', 'pennsylvania', 'rhode island', 'south carolina', 'south dakota', 'tennessee', 'texas',
    'utah', 'vermont', 'virginia', 'washington', ' DC ', 'west virginia', 'wisconsin', 'wyoming',
    'new york', 'ny', 'nyc', 'los angeles', 'chicago', 'houston', 'phoenix', 'philadelphia', 'san antonio', 'san diego',
    'dallas', 'san jose', 'austin', 'jacksonville', 'fort worth', 'columbus', 'indianapolis', 'charlotte', 'san francisco', 'seattle',
    'nashville', 'denver', 'oklahoma city', 'el paso', 'boston', 'portland', 'las vegas', 'vegas', 'detroit', 'memphis', 'louisville',
    'baltimore', 'milwaukee', 'albuquerque', 'tucson', 'fresno', 'sacramento', 'kansas city', 'mesa', 'atlanta', 'omaha',
    'colorado springs', 'raleigh', 'long beach', 'virginia beach', 'miami', 'oakland', 'minneapolis', 'tulsa', 'bakersfield',
    'wichita', 'arlington', 'aurora', 'tampa', 'new orleans', 'cleveland', 'honolulu', 'anaheim', 'lexington', 'stockton',
    'corpus christi', 'henderson', 'riverside', 'newark', 'st. paul', 'santa ana', 'cincinnati', 'irvine', 'orlando', 'pittsburgh',
    'st. louis', 'greensboro', 'jersey city', 'anchorage', 'lincoln', 'plano', 'durham', 'buffalo', 'chandler', 'chula vista',
    'toledo', 'madison', 'gilbert', 'reno', 'fort wayne', 'north las vegas', 'st. petersburg', 'lubbock', 'irving', 'laredo',
    'winston-salem', 'chesapeake', 'glendale', 'garland', 'scottsdale', 'norfolk', 'boise', 'fremont', 'spokane', 'santa clarita',
    'baton rouge', 'richmond', 'hialeah',
    'grand canyon', 'yellowstone', 'hollywood', 'niagara', 'disney world', 'yosemite', 'central park',
    'amtrak', 'greyhound', 'interstate', 'san', 'york'
]

# Initialize the TfidfVectorizer to calculate term frequency-inverse document frequency (TF-IDF)
# Use ngram_range=(1, 2) to consider both unigrams (1-word) and bigrams (2-words)
tfidf_vectorizer = TfidfVectorizer(stop_words=exclude_words, ngram_range=(1, 2))

# Fit the vectorizer on the documents and transform the text data
X_tfidf = tfidf_vectorizer.fit_transform(cleaned_texts_list)

# Get the list of words (features) for TF-IDF
tfidf_words = tfidf_vectorizer.get_feature_names_out()

# Convert the TF-IDF matrix into a DataFrame for better readability
tfidf_df = pd.DataFrame(X_tfidf.toarray(), index=cleaned_texts_list, columns=tfidf_words)

# Function to get top n words for each n-gram type (1-gram and 2-gram)
def get_top_n_grams(tfidf_df, n=10):
    # Sum the TF-IDF scores across all documents for each n-gram
    sum_tfidf = tfidf_df.sum(axis=0)

    # Sort the n-grams by their sum of TF-IDF values in descending order
    sorted_grams = sum_tfidf.sort_values(ascending=False)

    # Return the top n n-grams
    return sorted_grams.head(n)

# Filter unigrams (1-word) and bigrams (2-words)
unigrams = [word for word in tfidf_words if len(word.split()) == 1]
bigrams = [word for word in tfidf_words if len(word.split()) == 2]

# Get the top 10 unigrams (1-gram)
top_1grams = get_top_n_grams(tfidf_df[unigrams], n=15)
print("Top 15 Unigrams:")
print(top_1grams)

# Get the top 10 bigrams (2-gram)
top_2grams = get_top_n_grams(tfidf_df[bigrams], n=15)
print("\nTop 15 Bigrams:")
print(top_2grams)


/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['across', 'ana', 'angeles', 'antonio', 'baton', 'beach', 'canyon', 'carolina', 'central', 'christi', 'chula', 'city', 'clarita', 'corpus', 'dakota', 'dc', 'diego', 'disney', 'el', 'fort', 'francisco', 'grand', 'hampshire', 'island', 'jersey', 'jose', 'las', 'long', 'los', 'louis', 'mexico', 'new', 'north', 'orleans', 'park', 'paso', 'paul', 'petersburg', 'rhode', 'rouge', 'salem', 'santa', 'south', 'springs', 'st', 'states', 'the', 'united', 'us', 'vista', 'wayne', 'west', 'winston', 'world', 'worth'] not in stop_words.
  warnings.warn(


Top 15 Unigrams:
time      57.451589
day       56.442518
flight    52.878730
city      52.748550
place     51.841348
trip      51.107792
car       50.037077
want      47.499021
good      46.597402
think     41.017668
lot       39.575301
hotel     39.448018
people    38.350848
year      38.166532
fly       37.897906
dtype: float64

Top 15 Bigrams:
rent car                 16.328664
rental car               12.400245
car rental               11.360510
national park            10.192015
year ago                  9.594561
new orleans               8.945887
day trip                  8.753072
costa rica                7.577532
time year                 7.351130
west coast                7.338721
public transportation     6.764479
round trip                6.754204
road trip                 6.454885
east coast                6.444626
good luck                 6.202151
dtype: float64


Frugal before COVID

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np

# List of words to exclude
# car
# exclude_words = ['like', 'price', 'cost', 'inflation', 'deflation', 'expensive', 'cheap', 'purchase', 'sale']
# Real Estate
# exclude_words = ['real', 'estate', 'like', 'price', 'cost', 'inflation', 'deflation', 'expensive', 'cheap', 'purchase', 'sale']
# travel
# exclude_words = [
#     'travel', 'like', 'price', 'cost', 'inflation', 'deflation', 'expensive', 'cheap', 'purchase', 'sale',
#     'america', 'united states', 'usa', 'the us', 'u.s.', 'stateside', 'across the states',
#     'alabama', 'alaska', 'arizona', 'arkansas', 'california', 'colorado', 'connecticut', 'delaware', 'florida',
#     'georgia', 'hawaii', 'idaho', 'illinois', 'indiana', 'iowa', 'kansas', 'kentucky', 'louisiana', 'maine',
#     'maryland', 'massachusetts', 'michigan', 'minnesota', 'mississippi', 'missouri', 'montana', 'nebraska',
#     'nevada', 'new hampshire', 'new jersey', 'new mexico', 'new york', 'north carolina', 'north dakota', 'ohio',
#     'oklahoma', 'oregon', 'pennsylvania', 'rhode island', 'south carolina', 'south dakota', 'tennessee', 'texas',
#     'utah', 'vermont', 'virginia', 'washington', ' DC ', 'west virginia', 'wisconsin', 'wyoming',
#     'new york', 'ny', 'nyc', 'los angeles', 'chicago', 'houston', 'phoenix', 'philadelphia', 'san antonio', 'san diego',
#     'dallas', 'san jose', 'austin', 'jacksonville', 'fort worth', 'columbus', 'indianapolis', 'charlotte', 'san francisco', 'seattle',
#     'nashville', 'denver', 'oklahoma city', 'el paso', 'boston', 'portland', 'las vegas', 'vegas', 'detroit', 'memphis', 'louisville',
#     'baltimore', 'milwaukee', 'albuquerque', 'tucson', 'fresno', 'sacramento', 'kansas city', 'mesa', 'atlanta', 'omaha',
#     'colorado springs', 'raleigh', 'long beach', 'virginia beach', 'miami', 'oakland', 'minneapolis', 'tulsa', 'bakersfield',
#     'wichita', 'arlington', 'aurora', 'tampa', 'new orleans', 'cleveland', 'honolulu', 'anaheim', 'lexington', 'stockton',
#     'corpus christi', 'henderson', 'riverside', 'newark', 'st. paul', 'santa ana', 'cincinnati', 'irvine', 'orlando', 'pittsburgh',
#     'st. louis', 'greensboro', 'jersey city', 'anchorage', 'lincoln', 'plano', 'durham', 'buffalo', 'chandler', 'chula vista',
#     'toledo', 'madison', 'gilbert', 'reno', 'fort wayne', 'north las vegas', 'st. petersburg', 'lubbock', 'irving', 'laredo',
#     'winston-salem', 'chesapeake', 'glendale', 'garland', 'scottsdale', 'norfolk', 'boise', 'fremont', 'spokane', 'santa clarita',
#     'baton rouge', 'richmond', 'hialeah',
#     'grand canyon', 'yellowstone', 'hollywood', 'niagara', 'disney world', 'yosemite', 'central park',
#     'amtrak', 'greyhound', 'interstate', 'san', 'york'
# ]
exclude_words = ['frugal', 'like', 'price', 'cost', 'inflation', 'deflation', 'expensive', 'cheap', 'purchase', 'sale']

# Initialize the TfidfVectorizer to calculate term frequency-inverse document frequency (TF-IDF)
# Use ngram_range=(1, 2) to consider both unigrams (1-word) and bigrams (2-words)
tfidf_vectorizer = TfidfVectorizer(stop_words=exclude_words, ngram_range=(1, 2))

# Fit the vectorizer on the documents and transform the text data
X_tfidf = tfidf_vectorizer.fit_transform(cleaned_texts_list)

# Get the list of words (features) for TF-IDF
tfidf_words = tfidf_vectorizer.get_feature_names_out()

# Convert the TF-IDF matrix into a DataFrame for better readability
tfidf_df = pd.DataFrame(X_tfidf.toarray(), index=cleaned_texts_list, columns=tfidf_words)

# Function to get top n words for each n-gram type (1-gram and 2-gram)
def get_top_n_grams(tfidf_df, n=10):
    # Sum the TF-IDF scores across all documents for each n-gram
    sum_tfidf = tfidf_df.sum(axis=0)

    # Sort the n-grams by their sum of TF-IDF values in descending order
    sorted_grams = sum_tfidf.sort_values(ascending=False)

    # Return the top n n-grams
    return sorted_grams.head(n)

# Filter unigrams (1-word) and bigrams (2-words)
unigrams = [word for word in tfidf_words if len(word.split()) == 1]
bigrams = [word for word in tfidf_words if len(word.split()) == 2]

# Get the top 10 unigrams (1-gram)
top_1grams = get_top_n_grams(tfidf_df[unigrams], n=15)
print("Top 15 Unigrams:")
print(top_1grams)

# Get the top 10 bigrams (2-gram)
top_2grams = get_top_n_grams(tfidf_df[bigrams], n=15)
print("\nTop 15 Bigrams:")
print(top_2grams)


Top 15 Unigrams:
buy      147.903227
use      123.888450
car      114.176110
year     112.221775
good     109.400008
pay      107.138028
time     101.517401
look      94.463426
live      94.097667
money     91.027264
want      90.885079
month     90.704992
need      90.338385
thing     89.535526
find      89.297772
dtype: float64

Top 15 Bigrams:
save money       28.098878
grocery store    22.366959
year ago         18.651457
year old         15.268253
buy new          14.539080
good deal        13.025636
new car          11.538842
spend money      11.107503
pay month        10.535364
long time        10.522145
high quality     10.506620
lot money        10.108226
credit card       9.565503
thrift store      9.353152
find good         9.198424
dtype: float64


Frugal after COVID

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np

# List of words to exclude
# car
# exclude_words = ['like', 'price', 'cost', 'inflation', 'deflation', 'expensive', 'cheap', 'purchase', 'sale']
# Real Estate
# exclude_words = ['real', 'estate', 'like', 'price', 'cost', 'inflation', 'deflation', 'expensive', 'cheap', 'purchase', 'sale']
# travel
# exclude_words = [
#     'travel', 'like', 'price', 'cost', 'inflation', 'deflation', 'expensive', 'cheap', 'purchase', 'sale',
#     'america', 'united states', 'usa', 'the us', 'u.s.', 'stateside', 'across the states',
#     'alabama', 'alaska', 'arizona', 'arkansas', 'california', 'colorado', 'connecticut', 'delaware', 'florida',
#     'georgia', 'hawaii', 'idaho', 'illinois', 'indiana', 'iowa', 'kansas', 'kentucky', 'louisiana', 'maine',
#     'maryland', 'massachusetts', 'michigan', 'minnesota', 'mississippi', 'missouri', 'montana', 'nebraska',
#     'nevada', 'new hampshire', 'new jersey', 'new mexico', 'new york', 'north carolina', 'north dakota', 'ohio',
#     'oklahoma', 'oregon', 'pennsylvania', 'rhode island', 'south carolina', 'south dakota', 'tennessee', 'texas',
#     'utah', 'vermont', 'virginia', 'washington', ' DC ', 'west virginia', 'wisconsin', 'wyoming',
#     'new york', 'ny', 'nyc', 'los angeles', 'chicago', 'houston', 'phoenix', 'philadelphia', 'san antonio', 'san diego',
#     'dallas', 'san jose', 'austin', 'jacksonville', 'fort worth', 'columbus', 'indianapolis', 'charlotte', 'san francisco', 'seattle',
#     'nashville', 'denver', 'oklahoma city', 'el paso', 'boston', 'portland', 'las vegas', 'vegas', 'detroit', 'memphis', 'louisville',
#     'baltimore', 'milwaukee', 'albuquerque', 'tucson', 'fresno', 'sacramento', 'kansas city', 'mesa', 'atlanta', 'omaha',
#     'colorado springs', 'raleigh', 'long beach', 'virginia beach', 'miami', 'oakland', 'minneapolis', 'tulsa', 'bakersfield',
#     'wichita', 'arlington', 'aurora', 'tampa', 'new orleans', 'cleveland', 'honolulu', 'anaheim', 'lexington', 'stockton',
#     'corpus christi', 'henderson', 'riverside', 'newark', 'st. paul', 'santa ana', 'cincinnati', 'irvine', 'orlando', 'pittsburgh',
#     'st. louis', 'greensboro', 'jersey city', 'anchorage', 'lincoln', 'plano', 'durham', 'buffalo', 'chandler', 'chula vista',
#     'toledo', 'madison', 'gilbert', 'reno', 'fort wayne', 'north las vegas', 'st. petersburg', 'lubbock', 'irving', 'laredo',
#     'winston-salem', 'chesapeake', 'glendale', 'garland', 'scottsdale', 'norfolk', 'boise', 'fremont', 'spokane', 'santa clarita',
#     'baton rouge', 'richmond', 'hialeah',
#     'grand canyon', 'yellowstone', 'hollywood', 'niagara', 'disney world', 'yosemite', 'central park',
#     'amtrak', 'greyhound', 'interstate', 'san', 'york'
# ]
exclude_words = ['frugal', 'like', 'price', 'cost', 'inflation', 'deflation', 'expensive', 'cheap', 'purchase', 'sale']

# Initialize the TfidfVectorizer to calculate term frequency-inverse document frequency (TF-IDF)
# Use ngram_range=(1, 2) to consider both unigrams (1-word) and bigrams (2-words)
tfidf_vectorizer = TfidfVectorizer(stop_words=exclude_words, ngram_range=(1, 2))

# Fit the vectorizer on the documents and transform the text data
X_tfidf = tfidf_vectorizer.fit_transform(cleaned_texts_list)

# Get the list of words (features) for TF-IDF
tfidf_words = tfidf_vectorizer.get_feature_names_out()

# Convert the TF-IDF matrix into a DataFrame for better readability
tfidf_df = pd.DataFrame(X_tfidf.toarray(), index=cleaned_texts_list, columns=tfidf_words)

# Function to get top n words for each n-gram type (1-gram and 2-gram)
def get_top_n_grams(tfidf_df, n=10):
    # Sum the TF-IDF scores across all documents for each n-gram
    sum_tfidf = tfidf_df.sum(axis=0)

    # Sort the n-grams by their sum of TF-IDF values in descending order
    sorted_grams = sum_tfidf.sort_values(ascending=False)

    # Return the top n n-grams
    return sorted_grams.head(n)

# Filter unigrams (1-word) and bigrams (2-words)
unigrams = [word for word in tfidf_words if len(word.split()) == 1]
bigrams = [word for word in tfidf_words if len(word.split()) == 2]

# Get the top 10 unigrams (1-gram)
top_1grams = get_top_n_grams(tfidf_df[unigrams], n=15)
print("Top 15 Unigrams:")
print(top_1grams)

# Get the top 10 bigrams (2-gram)
top_2grams = get_top_n_grams(tfidf_df[bigrams], n=15)
print("\nTop 15 Bigrams:")
print(top_2grams)


Top 15 Unigrams:
buy      55.607722
use      43.519472
year     42.682866
car      40.023208
good     36.893904
time     33.822442
look     33.532132
pay      33.163801
food     32.982746
thing    30.605548
think    30.072742
need     29.853243
store    29.755777
lot      29.478572
know     29.245842
dtype: float64

Top 15 Bigrams:
year ago         8.007792
save money       7.656282
grocery store    7.266219
buy new          7.169520
year old         5.991183
use car          5.750606
new car          5.091808
thrift store     4.300162
buy thing        4.232861
spend money      4.116472
good deal        4.104385
lot money        3.817864
long run         3.779976
high quality     3.629798
buy house        3.434851
dtype: float64
